# Dihedral symmetry of the Robinson form

**Adapted from**: Example 5.4 of [GP04]

[GP04] Gatermann, Karin and Parrilo, Pablo A.
*Symmetry groups, semidefinite programs, and sums of squares*.
Journal of Pure and Applied Algebra 192.1-3 (2004): 95-128.

Symmetry reduction is still a work in progress in SumOfSquares, so we include the following files that will be incorporated into SumOfSquares.jl once SymbolicWedderburn.jl is released:

In [1]:
using SumOfSquares
include(joinpath(dirname(dirname(pathof(SumOfSquares))), "examples", "symmetry.jl"))
include(joinpath(dirname(dirname(pathof(SumOfSquares))), "examples", "scaled_perm.jl"))

    Cloning git-repo `https://github.com/kalmarek/SymbolicWedderburn.jl`
   Updating git-repo `https://github.com/kalmarek/SymbolicWedderburn.jl`
   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
Updating `~/work/SumOfSquares.jl/SumOfSquares.jl/docs/Project.toml`
  [858aa9a9] + SymbolicWedderburn v0.1.0 `https://github.com/kalmarek/SymbolicWedderburn.jl#bl/nonperm`
Updating `~/work/SumOfSquares.jl/SumOfSquares.jl/docs/Manifest.toml`
  [858aa9a9] + SymbolicWedderburn v0.1.0 `https://github.com/kalmarek/SymbolicWedderburn.jl#bl/nonperm`


We start by defining the Dihedral group of order 8.
This group is isomorphic to the following permutation group:

In [2]:
d = perm"(1, 2, 3, 4)"
c = perm"(1, 3)"
G = PermGroup([c, d])

Permutation group on 2 generators generated by
 (1,3)
 (1,2,3,4)

We could rely on this isomorphism to define this group.
However, in order to illustrate how to do symmetry reduction with a custom group,
we show in this example what should be implemented to define a new group.

In [3]:
struct DihedralElement <: GroupElem
    n::Int
    reflection::Bool
    id::Int
end
function PermutationGroups.order(el::DihedralElement)
    if el.reflection
        return 2
    else
        if iszero(el.id)
            return 1
        else
            return div(el.n, gcd(el.n, el.id))
        end
    end
end
Base.one(el::Union{DihedralElement}) = DihedralElement(el.n, false, 0)
function Base.inv(el::DihedralElement)
    if el.reflection || iszero(el.id)
        return el
    else
        return DihedralElement(el.n, false, el.n - el.id)
    end
end
function Base.:*(a::DihedralElement, b::DihedralElement)
    a.n == b.n || error("Cannot multiply elements from different Dihedral groups")
    id = mod(a.reflection ? a.id - b.id : a.id + b.id, a.n)
    return DihedralElement(a.n, a.reflection != b.reflection, id)
end
function PermutationGroups.mul!(::DihedralElement, a::DihedralElement, b::DihedralElement)
    return a * b
end
function Base.:^(el::DihedralElement, k::Integer)
    if el.reflection
        return iseven(k) ? one(el) : el
    else
        return DihedralElement(el.n, false, mod(el.id * k, el.n))
    end
end

Base.conj(a::DihedralElement, b::DihedralElement) = inv(b) * a * b
Base.:^(a::DihedralElement, b::DihedralElement) = conj(a, b)

struct DihedralGroup <: Group
    n::Int
end
Base.one(G::DihedralGroup) = DihedralElement(G.n, false, 0)
PermutationGroups.gens(G::DihedralGroup) = [DihedralElement(G.n, false, 1), DihedralElement(G.n, true, 0)]
PermutationGroups.order(::Type{T}, G::DihedralGroup) where {T} = convert(T, 2G.n)
function Base.iterate(G::DihedralGroup, prev::DihedralElement=DihedralElement(G.n, false, -1))
    if prev.id + 1 >= G.n
        if prev.reflection
            return nothing
        else
            next = DihedralElement(G.n, true, 0)
        end
    else
        next = DihedralElement(G.n, prev.reflection, prev.id + 1)
    end
    return next, next
end

The Robinson form is invariant under the following action of the Dihedral group on monomials:
The action of each element of the groups is to map the variables `x, y` to:

| id | rotation | reflection |
|----|----------|------------|
| 0  | x, y     | y, x       |
| 1  | -y, x    | -x, y      |
| 2  | -x, -y   | -y, -x     |
| 3  | y, -x    | x, -y      |

In [4]:
using DynamicPolynomials
@polyvar x y
function action(mono::MP.AbstractMonomial, el::DihedralElement)
    if iseven(el.reflection + el.id)
        var_x, var_y = x, y
    else
        var_x, var_y = y, x
    end
    sign_x = 1 <= el.id <= 2 ? -1 : 1
    sign_y = 2 <= el.id ? -1 : 1
    return MP.substitute(MP.Eval(), mono, [x, y] => [sign_x * var_x, sign_y * var_y])
end
function action(term::MP.AbstractTerm, el::DihedralElement)
    return MP.coefficient(term) * action(MP.monomial(term), el)
end
function action(poly::MP.AbstractPolynomial, el::DihedralElement)
    return MP.polynomial([action(term, el) for term in MP.terms(poly)])
end

poly = x^6 + y^6 - x^4 * y^2 - y^4 * x^2 - x^4 - y^4 - x^2 - y^2 + 3x^2 * y^2 + 1

x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1

We can verify that `poly` is indeed invariant under the action of each element of the group as follows.

In [5]:
G = DihedralGroup(4)
for g in G
    @show action(poly, g)
end

action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1


We can exploit this symmetry for reducing the problem using the `SymmetricIdeal` certificate as follows:

In [6]:
import CSDP
function solve(G)
    solver = CSDP.Optimizer
    model = Model(solver)
    @variable(model, t)
    @objective(model, Max, t)
    con_ref = @constraint(model, poly - t in SOSCone(), ideal_certificate = SymmetricIdeal(SOSCone(), G, action))
    optimize!(model)
    @show value(t)


    for g in gram_matrix(con_ref).sub_gram_matrices
        println(g.basis.polynomials)
    end
end
solve(G)

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.03e-01 Pobj: -4.5636299e+00 Ad: 8.45e-01 Dobj:  5.2967876e+00 
Iter:  2 Ap: 7.31e-01 Pobj: -1.1835354e+01 Ad: 8.76e-01 Dobj:  1.4675311e+01 
Iter:  3 Ap: 8.91e-01 Pobj: -6.8243431e+00 Ad: 7.62e-01 Dobj:  7.6214135e+00 
Iter:  4 Ap: 8.67e-01 Pobj: -2.7355189e+00 Ad: 8.59e-01 Dobj:  1.2146327e+00 
Iter:  5 Ap: 8.79e-01 Pobj: -1.5458374e+00 Ad: 8.36e-01 Dobj: -3.5074262e-01 
Iter:  6 Ap: 7.96e-01 Pobj: -1.1190240e+00 Ad: 7.94e-01 Dobj: -7.3112750e-01 
Iter:  7 Ap: 8.52e-01 Pobj: -9.9963334e-01 Ad: 7.39e-01 Dobj: -8.5858732e-01 
Iter:  8 Ap: 7.76e-01 Pobj: -9.5570165e-01 Ad: 7.75e-01 Dobj: -9.0656574e-01 
Iter:  9 Ap: 8.61e-01 Pobj: -9.4215903e-01 Ad: 7.57e-01 Dobj: -9.2437049e-01 
Iter: 10 Ap: 7.84e-01 Pobj: -9.3679565e-01 Ad: 8.32e-01 Dobj: -9.3072397e-01 
Iter: 11 Ap: 8.30e-01 Pobj: -9.3508080e-01 Ad: 8.13e-01 Dobj: -9.3273624e-01 
Iter: 12 Ap: 7.95e-01 Pobj: -9.3455269e-01 Ad: 8.23e-

We notice that we indeed find `-3825/4096` and that symmetry was exploited.
In case the conjugacy classes are known, we can implement
`SymbolicWedderburn.conjugacy_classes_orbit` instead of `gens`, `order` and `iterate`.
To show that these do not need to be implemented, we create a new dihedral group type
that do not implement these methods but that instead implement
`SymbolicWedderburn.conjugacy_classes_orbit`:

In [7]:
struct DihedralGroup2 <: Group
    n::Int
end
_orbit(cc::Vector{<:GroupElem}) = PermutationGroups.Orbit(cc, Dict(a => nothing for a in cc))
_orbit(el::GroupElem) = _orbit([el])
function SymbolicWedderburn.conjugacy_classes_orbit(d::DihedralGroup2)
    orbits = [_orbit(DihedralElement(d.n, false, 0))]
    for i in 1:div(d.n - 1, 2)
        push!(orbits, _orbit([
            DihedralElement(d.n, false, i),
            DihedralElement(d.n, false, d.n - i),
        ]))
    end
    if iseven(d.n)
        push!(orbits, _orbit(DihedralElement(d.n, false, div(d.n, 2))))
        push!(orbits, _orbit([
            DihedralElement(d.n, true, i) for i in 0:2:(d.n - 2)
        ]))
        push!(orbits, _orbit([
            DihedralElement(d.n, true, i) for i in 1:2:(d.n - 1)
        ]))
    else
        push!(orbits, _orbit([
            DihedralElement(d.n, true, i) for i in 0:(d.n - 1)
        ]))
    end
end

As we have not implemented the iterator over all the elements, we can iterate over all
conjugacy classes instead to verify that the polynomial is invariant under the group action.

In [8]:
G = DihedralGroup2(4)
for cc in SymbolicWedderburn.conjugacy_classes_orbit(G)
    for g in cc
        @show action(poly, g)
    end
end

solve(G)

action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
Iter: 18 Ap: 7.76e-01 Pobj: -9.3338347e-01 Ad: 9.59e-01 Dobj: -9.3361059e-01 
Success: SDP solved
Primal objective value: -9.3338347e-01 
Dual objective value: -9.3361059e-01 
Relative primal infeasibility: 2.86e-09 
Relative dual infeasibility: 2.80e-10 
Real Relative Gap: -7.92e-05 
XZ Relative Gap: 2.32e-09 
DIMACS error measures: 3.75e-09 0.00e+00 7.11e-10 0.00e+00 -7.92e-05 2.32e-09
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*